# AOC2019 - Day 15

Today's challenge: [Day 15](https://adventofcode.com/2019/day/15)

Data are manually downloaded into `./data/input_15.txt`

In [1]:
with open('./data/input_15.txt') as f:
    raw = f.read()

In [2]:
class Intcode:
    def __init__(self, xs, pos=0, i=None):
        self.xs = xs
        self.pos = pos
        self.rel_base = 0
        self.i = i
        self.res = 0
        
    def display(self):
        return dict(enumerate(self.xs))
    
    @staticmethod
    def parse_op(x):
        params, op = divmod(x, 100)
        params, p1 = divmod(params, 10)
        params, p2 = divmod(params, 10)
        p3 = params % 10
        return op, p1, p2, p3
    
    def get_value(self,i,p):
        if p == 0:
            return self.xs[i]   
        elif p == 1:
            return i
        elif p == 2:
            i_ = i + self.rel_base
            return self.xs[i_]
        else:
            print("WTF:", p, i)
    
    def set_value(self,i,p):
        if p == 0:
            return i
        elif p == 2:
            i_ = i + self.rel_base
            return i_
        else:
            print("WTF:", p, i)
    
    def op1(self,i1,i2,out,ps):
        p1, p2, pout = ps
        i1_ = self.get_value(i1, p1)
        i2_ = self.get_value(i2, p2)
        out_ = self.set_value(out, pout)
        self.xs[out_] = i1_ + i2_
        return 4
    
    def op2(self,i1,i2,out,ps):
        p1, p2, pout = ps
        i1_ = self.get_value(i1, p1)
        i2_ = self.get_value(i2, p2)
        out_ = self.set_value(out, pout)
        self.xs[out_] = i1_ * i2_
        return 4

    def op3(self,x,i,ps):
        p, *_ = ps
        i_ = self.set_value(i, p)
        self.xs[i_] = x
        return 2

    def op4(self,i,ps):
        p, *_ = ps
        i_ = self.get_value(i, p)
        self.res = i_
        if i_ == -1:
            pass
            #print("TEMP: ", [self.xs[x] for x in [2391,2392,2394]])
        return 2
    
    def op5(self,i,out,ps):
        pi, pout, _ = ps
        i_ = self.get_value(i, pi)
        #out_ = self.set_value(out, pout)
        out_ = self.get_value(out, pout)
        if i_ != 0:
            self.pos = out_
            return 0
        return 3
    
    def op6(self,i,out,ps):
        pi, pout, _ = ps
        i_ = self.get_value(i, pi)
        out_ = self.get_value(out, pout)
        if i_ == 0:
            self.pos = out_
            return 0
        return 3
    
    def op7(self,i1,i2,out,ps):
        p1, p2, pout = ps
        i1_ = self.get_value(i1, p1)
        i2_ = self.get_value(i2, p2)
        out_ = self.set_value(out, pout)
        self.xs[out_] = 1 if i1_ < i2_ else 0
        return 4
    
    def op8(self,i1,i2,out,ps):
        p1, p2, pout = ps
        i1_ = self.get_value(i1, p1)
        i2_ = self.get_value(i2, p2)
        out_ = self.set_value(out, pout)
        self.xs[out_] = 1 if i1_ == i2_ else 0
        return 4
    
    def op9(self,i,ps):
        p, *_ = ps
        i_ = self.get_value(i, p)
        self.rel_base += i_
        return 2
    
    def op99(self):
        return 0

    def run(self, x=0):
        while True:
            op, *params = self.parse_op(self.xs[self.pos])
            if self.pos == 433:
                pass
                #print(op, self.pos, self.rel_base, self.xs[2394])
            if op%100 == 1:
                i1, i2, out = [self.xs[x] for x in range(self.pos+1,self.pos+4)]
                self.pos += self.op1(i1, i2, out, params)
            elif op%100 == 2:
                i1, i2, out = [self.xs[x] for x in range(self.pos+1,self.pos+4)]
                self.pos += self.op2(i1, i2, out, params)
            elif op%100 == 3:
                i = self.xs[self.pos+1]
                self.pos += self.op3(x, i, params)
            elif op%100 == 4:
                i = self.xs[self.pos+1]
                self.pos += self.op4(i, params)
                return self.res, True
            elif op%100 == 5:
                i, out = [self.xs[x] for x in range(self.pos+1,self.pos+3)]
                p = self.op5(i, out, params)
                self.pos += p
            elif op%100 == 6:
                i, out = [self.xs[x] for x in range(self.pos+1,self.pos+3)]
                p = self.op6(i, out, params)
                self.pos += p
            elif op%100 == 7:
                i1, i2, out = [self.xs[x] for x in range(self.pos+1,self.pos+4)]
                self.pos += self.op7(i1, i2, out, params)
            elif op%100 == 8:
                i1, i2, out = [self.xs[x] for x in range(self.pos+1,self.pos+4)]
                self.pos += self.op8(i1, i2, out, params)
            elif op%100 == 9:
                i1 = self.xs[self.pos+1]
                self.pos += self.op9(i1, params)
            elif op%100 == 99:
                self.op99()
                return self.res, False
            else:
                print("invalid instruction!", op)
                break

In [383]:
class RC:
    def __init__(self, brain):
        self.p = brain
        self.x = 0
        self.y = 0
        self.panel = defaultdict(lambda : " ")
        self.panel[(0,0)] = 'o'
        self.d_pos = 0
        self.directions = [1,4,2,3]
        self.d = 1
        self.target = (None, None)
        
    def __repr__(self):
        xs, ys = zip(*self.panel.keys())
        x0, x1 = min(xs), max(xs)
        y0, y1 = min(ys), max(ys)
        res = []
        for y in range(y0, y1+1):
            row = ""
            for x in range(x0,x1+1):
                if (x,y) == (0,0):
                    row += 'S'
                elif (x, y) == self.target:
                    row += 'T'
                elif self.panel[(x,y)] == 'o':
                    row += '.'
                else:
                    row += self.panel[(x,y)]
            res.append(row)
        return "\n".join(res)
    
    def move(self, nm):
        self.panel[(self.x, self.y)] = '.'
        if nm == 1: #N
            self.y -= 1
        elif nm == 2: #S
            self.y += 1
        elif nm == 3: #W
            self.x -= 1
        elif nm == 4: #E
            self.x += 1
        self.panel[(self.x, self.y)] = 'o'

    def check_next(self, nm):
        if nm == 1: #N
            return self.panel[(self.x, self.y-1)]
        elif nm == 2: #S
            return self.panel[(self.x, self.y+1)]
        elif nm == 3: #W
            return self.panel[(self.x-1, self.y)]
        elif nm == 4: #E
            return self.panel[(self.x+1, self.y)]
        
    def rotate(self, a):
        self.d_pos = (self.d_pos + 4 + a) % 4
        self.d = self.directions[self.d_pos]
                    
    def draw(self, nm, s):
        if nm == 1: #N
            self.panel[(self.x, self.y-1)] = s
        elif nm == 2: #S
            self.panel[(self.x, self.y+1)] = s
        elif nm == 3: #W
            self.panel[(self.x-1, self.y)] = s
        elif nm == 4: #E
            self.panel[(self.x+1, self.y)] = s
        if s == 'T':
            print(self)
            
    def run(self, xs=[]):
        c = True
        step = 0
        while c:
            i = None
            while not i in ['1','2','3','4','c']:
                i = input()
            if i == 'c':
                break
            next_move = int(i)
            out, c = self.p.run(next_move)
            if out == 0:
                self.draw(next_move,'#')
            elif out == 1:
                self.move(next_move)
                step += 1
            elif out == 2:
                print("found! %s, %s, while moving %s" % (self.x, self.y, next_move))
            print(out)
            print(self)
            print(80*'-')
    
    def autorun(self, xs=[]):
        MAX = 5700
        c = True
        step = 0
        while c:
            if step > MAX:
                break
            step += 1
            i = self.d
            next_move = i
            out, c = self.p.run(next_move)
            if out == 0:
                self.draw(next_move, '#')
                self.rotate(-1)
            elif out == 1 or out == 2:
                self.move(next_move)
                self.rotate(1)
                if out == 2:
                    print("%s: found! %s, %s" % (step, self.x, self.y))
                    self.target = (self.x, self.y)
            with open('lab.txt','w') as f:
                f.write(str(self))

In [384]:
from collections import defaultdict, Counter
data = defaultdict(int)
data.update({(i,int(x)) for i,x in enumerate(raw.strip().split(","))})

In [385]:
p = Intcode(data)
remote = RC(p)

In [386]:
remote.autorun()

2431: found! -16, -12
5627: found! -16, -12


In [387]:
with open("lab.txt") as f:
    lab = [s[:-1] for s in f.readlines()]
lab

[' ########### ######### ######### ####### ',
 '#...........#.........#.........#.......#',
 '#.#####.#####.#######.###.#.#####.###.#.#',
 '#.#.....#.....#.....#...#.#.........#.#.#',
 '#.#.#####.#######.#.###.#.#########.#.## ',
 '#.#.#.....#.#.....#.#...#.....#...#.#...#',
 '#.###.#####.#.#.#####.#########.#.#####.#',
 '#.#...#.#...#.#.....#.....#.....#.......#',
 '#.#.###.#.#.#.#####.#####.#.#####.#####.#',
 '#.#.#T#...#.#.#...#.#.....#.....#.#...#.#',
 '#.#.#.#.###.#.###.#.#.#####.###.###.#.#.#',
 '#...#.....#.......#.#.#.......#.....#.#.#',
 '#.###.#############.#.#######.#######.#.#',
 '#.#...#...........#.#.....#.#.....#...#.#',
 '#.###.#.#.#######.#.#####.#.###.###.###.#',
 '#...#.#.#.#...#.#...#.....#.....#...#.#.#',
 ' ##.#.#.###.#.#.###.#.#######.###.###.#.#',
 '#...#.#.....#.#.....#...#...#.#...#...#.#',
 '#.#####.#####.#######.#.#.#.#.#.#####.#.#',
 '#.....#.#.....#.....#.#...#.#.#.#.....#.#',
 ' ####.#.#.#.###.###.###.###.###.#.#####.#',
 '#.#...#.#.#.#...#.#..S#.#.#...#.

In [388]:
import networkx as nx

In [389]:
g_lab = nx.Graph()

for y, row in enumerate(lab[:-1]):
    for x, c in enumerate(row[:-1]):
        if c == '#' or c == ' ':
            continue
        if c == 'T':
            print("T", x, y)
        if c == 'S':
            print("S", x, y)
        if row[x+1] in ['S', 'T', '.', 'o']:
            g_lab.add_edge((x,y), (x+1,y))
        if lab[y+1][x] in ['S', 'T', '.', 'o']:
            g_lab.add_edge((x,y), (x,y+1))

T 5 9
S 21 21


In [390]:
nx.shortest_path_length(g_lab, (5,9), (21, 21))

240

In [391]:
max(nx.shortest_path_length(g_lab, (5,9)).values())

322